# Trained model Details

In [39]:
import tensorflow as tf
from tensorflow.keras.utils import plot_model
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator

## Read the model

1. Model to classify category

In [40]:
model_category=tf.keras.models.load_model(r"C:\Users\amitu\Desktop\fashion_model\trained model\model_category.h5")

In [41]:
#summary of the model
model_category.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
 dense (Dense)               (None, 128)               262272    
                                                                 
 batch_normalization (Batch  (None, 128)               512       
 Normalization)                                                  
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 batch_normalization_1 (Bat  (None, 64)                256       
 chNormalization)                                       

2. Model to classify gender

In [42]:
model_gender= tf.keras.models.load_model(r"C:\Users\amitu\Desktop\New folder\my_model_gender.h5")

In [43]:
#summary of the model
model_gender.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
 dense_4 (Dense)             (None, 64)                131136    
                                                                 
 batch_normalization_3 (Bat  (None, 64)                256       
 chNormalization)                                                
                                                                 
 dropout_3 (Dropout)         (None, 64)                0         
                                                                 
 dense_5 (Dense)             (None, 32)                2080      
                                                                 
 batch_normalization_4 (Bat  (None, 32)                128       
 chNormalization)                                     

#### Read data frame 

In [44]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
## Read Train data CSV file
df_train=pd.read_csv(r"path\train_data.csv")
## Read Test data
df_test=pd.read_csv(r"path\test_data.csv")

In [ ]:
df_train = pd.read_csv(r'path\train_data.csv',error_bad_lines=False)
df_train['image'] = df_train.apply(lambda row: str(row['id']) + ".jpg", axis=1)

In [ ]:
df_test = pd.read_csv(r'path\test_data.csv',error_bad_lines=False)
df_test['image'] = df_test.apply(lambda row: str(row['id']) + ".jpg", axis=1)

### Accuracy of Trained model

#### Read image using genator

In [47]:
# Data augmentation with different settings
image_generator = ImageDataGenerator(
    rescale=1.0 / 255.0,  # Normalize pixel values to [0, 1]
    rotation_range=15,   # Random rotation
    width_shift_range=0.1,  # Random horizontal shift
    height_shift_range=0.1,  # Random vertical shift
    shear_range=0.1,  # Shear transformation
    zoom_range=0.1,  # Random zoom
    horizontal_flip=True,  # Random horizontal flip
    fill_mode='nearest'
)
####################################
# count of each class
class_counts = df_test['articleType'].value_counts().to_dict()
#####################################
# Class weights for balancing
class_weights = {class_idx: max(class_counts.values()) / count for class_idx, count in class_counts.items()}

In [ ]:
# Flow training data for article type
training_generator = image_generator.flow_from_dataframe(
    dataframe=df_train,
    directory=r"path\train_image",
    x_col="image",
    y_col="articleType",
    target_size=(200,200),
    batch_size=32,
    class_mode="categorical", 
    shuffle=True,
    class_weights=class_weights  
)

In [ ]:
# Flow training data for gender
training_generator_gender = image_generator.flow_from_dataframe(
    dataframe=df_train,
    directory=r"path\train_image",
    x_col="image",
    y_col="gender",
    target_size=(200,200),
    batch_size=32,
    class_mode="categorical", 
    shuffle=True,
    class_weights=class_weights  
)

In [ ]:
# validation data for article type
validation_generator = image_generator.flow_from_dataframe(
    dataframe=df_test,
    directory=r"path\test_image",
    x_col="image",
    y_col="articleType",
    target_size=(200,200),
    batch_size=32,
    class_mode="categorical",  # Use "categorical" for one-hot encoded labels
    shuffle=False
)

In [ ]:
# validation data for article type
validation_generator_gender= image_generator.flow_from_dataframe(
    dataframe=df_test,
    directory=r"path\test_image",
    x_col="image",
    y_col="gender",
    target_size=(200,200),
    batch_size=32,
    class_mode="categorical", 
    shuffle=False
)

#### Accuracy of model

#### 1. Article type classification Model

In [56]:
accuracy = model_category.evaluate(validation_generator)
print('Accuracy:', accuracy)

134/134 [==============================] - 816s 6s/step - loss: 0.5183 - accuracy: 0.9206
Accuracy: [0.5183279514312744, 0.920634925365448]


#### 2. Gender classification model

In [60]:
accuracy_gender= model_gender.evaluate(validation_generator_gender)
print('Accuracy:', accuracy_gender)

134/134 [==============================] - 827s 6s/step - loss: 0.1867 - accuracy: 0.9314
Accuracy: [0.18666262924671173, 0.9313725233078003]
